---
jupyter:
  jupytext:
    formats: md,ipynb
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="fd757e24" -->
# Table des matières
1. [Bon à savoir](#bon-à-savoir)
1. [Lecture des fichiers de données](#lecture-des-fichiers-de-données)
1. [Génération de données multidimensionnelles à partir de la matrice des distances](#génération-de-données-multidimensionnelles-à-partir-de-la-matrice-des-distances)
1. [Transformation PCA](#transformation-pca)
  1. [Sélection du nombre optimal de dimensions](#sélection-du-nombre-optimal-de-dimensions)
1. [Affichage des positions 2-D des villes](#affichage-des-positions-2-d-des-villes)
1. [Y a-t-il d'autres applications possibles?](#y-a-t-il-dautres-applications-possibles)
<!-- #endregion -->



In [ ]:
%matplotlib inline

import os
from math import asin, cos, pi, sqrt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.preprocessing import StandardScaler

sns.set(color_codes=True)

seed = 42
np.random.seed(seed)



<!-- #region id="f00c86a0" -->
Dans un module précédent, on a vu qu'on pouvait réduire la dimensionnalité d'un jeu de données $X$ afin de créer de nouvelles
caractéristiques permettant de visualiser plus facilement des données. Dans l'exemple utilisant la base
de données **MNIST**, on passait ainsi d'un ensemble de données en 64-D vers un nouvel ensemble en 2-D.
Beaucoup d'algorithmes de réduction de la dimensionnalité utilisent la matrice des distances entre les
différents points lors des calculs.

La méthode de réduction de la dimensionnalité MDS (*Multidimensional Scaling*) a une particularité assez intéressante.
Même si l'on a perdu de l'information lors de la réduction de dimensionnalité, elle permet
de retrouver approximativement les données initiales X!

Pourquoi voudrions-nous faire cela? Comme on dit souvent « une image vaut mille mots ». Dans ce
module, nous allons prédire les positions originales de villes canadiennes et américaines uniquement à partir de leur
matrice des distances. Ces distances ont été originellement calculées à partir de la latitude et de la longitude
de chaque ville. Les distances sont donc assez près des distances de vol observées en réalité. C'est un
exemple classique d'utilisation de la méthode MDS.

<!-- #endregion -->

<!-- #region id="da7ff45b" -->
# <a id=bon-à-savoir>Bon à savoir</a>
<!-- #endregion -->

<!-- #region id="314d03f9" -->
On utilise dans ce module
l'[analyse en composantes principales](https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales)
afin d’extraire des données générées par la méthode MDS. Il serait trop long d’expliquer
ici comment les méthodes MDS et PCA fonctionnent. Si vous voulez en savoir plus, nous vous invitons à lire à leur
sujet en consultant par exemple la documentation de Scikit-learn.

Tel que mentionné précédemment dans le module sur la réduction de la dimensionnalité des données, on essaie souvent
plusieurs méthodes pour solutionner un problème. Celles qui fonctionnent bien sont ensuite analysées
plus à fond. Pas besoin, dans un premier temps, de connaître à fond la théorie de
chaque méthode possiblement intéressante. C’est un peu l’idée de Scikit-learn qui montre plein
de démos utiles. On essaie celles qui semblent prometteuses et on approfondit ensuite ses connaissances
sur celles qu’on a finalement décidé d’utiliser.

<!-- #endregion -->

<!-- #region id="a950347b" -->
# <a id=lecture-des-fichiers-de-données>Lecture des fichiers de données</a>
<!-- #endregion -->

<!-- #region id="a472203d" -->
Nous utilisons deux fichiers de données contenant les noms de 312 villes canadiennes et américaines, ainsi
que les distances entre elles. Les données proviennent du répertoire
[CITIES]: https://people.sc.fsu.edu/~jburkardt/datasets/cities/cities.html qui
contient différents ensembles de données sur les distances entre les villes.
<!-- #endregion -->



In [ ]:
# Lecture du fichier contenant les noms des villes

N_villes = 312

# Provinces et territoires au Canada
Canada = ["AB", "BC", "MB", "NB", "NF", "NS", "NT", "ON", "PE", "QC", "SK", "YT"]

file = open("/pax/shared/GIF-U014/usca312_name.txt", "r")
villes = []
pays = []
for line in file:
    # Les premières lignes contiennent des commentaires; on les ignore.
    if not line.startswith("#"):
        # Séparation des noms de villes et des états (ou provinces,
        # territoires, protectorats)
        [ville, etat] = line.split(",")
        etat = etat.strip("\n").strip(" ")

        # Identification des villes canadiennes
        if etat in Canada:
            pays.append(True)
        else:
            pays.append(False)

        villes.append(ville)


In [ ]:
# Lecture du fichier contenant les distances (en miles) entre les villes

line2 = []
file = open("/pax/shared/GIF-U014/usca312_dist.txt", "r")
for line in file:
    # Les premières lignes contiennent des commentaires; on les ignore.
    if not line.startswith("#"):
        line = line.replace("\n", " ").split()
        line2 = line2 + line

line2 = np.array(line2, float)

# Combien y a-t-il de valeurs?
print(line2.shape[0])



<!-- #region id="8857bac1" -->
Cela correspond bien à la taille d'une matrice de distances entre 312 points, soit $312 \times 312 = 97 344$.
<!-- #endregion -->



In [ ]:
# Mise en forme de la matrice des distances
distances = np.reshape(line2, (N_villes, N_villes))

# Conversion des distances de miles en kilomètres (on est au
# vingt-et-unième siècle après tout!)
distances = distances * 1.6094



<!-- #region id="b1f2275e" -->
# <a id=génération-de-données-multidimensionnelles-à-partir-de-la-matrice-des-distances>Génération de données multidimensionnelles à partir de la matrice des distances</a>
<!-- #endregion -->

<!-- #region id="23fcbe74" -->
On va générer un ensemble de 312 données multidimensionnelles pour lequel les distances interobservations
reproduisent la matrice originale des distances.

Puisqu'on ne sait pas *a priori* la dimension de l'espace à utiliser, on va générer les
données X dans un espace de dimension 10 puis utiliser l'analyse en composantes principales
pour nous dire combien de dimensions sont réellement nécessaires (moins de 10).

<!-- #endregion -->



In [ ]:
embedding = MDS(n_components=10, metric=True, dissimilarity="precomputed")
X = embedding.fit_transform(distances)

# Les observations X forment une matrice de dimensions 312 x 10  (312 villes
# en 10 dimensions)
print(X.shape)



<!-- #region id="90f2b21c" -->
# <a id=transformation-pca>Transformation PCA</a>
<!-- #endregion -->

<!-- #region id="d049b5dc" -->
Contrairement à ce que l'on vous dit toujours de faire, on ne va pas normaliser les valeurs de X avant de
calculer la transformation. On veut que les nouvelles coordonnées transformées utilisent encore des unités naturelles, en km.
<!-- #endregion -->



In [ ]:
pca = PCA()
_ = pca.fit(X)



<!-- #region id="ead1a573" -->
## <a id=sélection-du-nombre-optimal-de-dimensions>Sélection du nombre optimal de dimensions</a>
<!-- #endregion -->

<!-- #region id="3af941a4" -->
L'analyse en composantes principales est généralement utilisée pour déterminer le nombre de dimensions importantes dans
un ensemble de données multidimensionnelles. On utilise à cette fin le diagramme d'éboulis (*Scree Plot*) qui montre
la variance $\lambda$ de chaque composante. Cela correspond, en gros, à la quantité d'information associée à
chaque dimension de la PCA. Dans le diagramme en éboulis, cette quantité d'information est affichée en ordre décroissant d'importance.

La figure suivante montre qu'il y a deux composantes vraiment importantes et une troisième à la limite du négligeable.
Les autres contiennent essentiellement du bruit et peuvent être négligées.

À quoi correspondent ces trois premières composantes? Ce sont approximativement les coordonnées x, y et z des villes
sur la Terre! Les valeurs de la troisième composante, l'altitude z, demeurent faibles par rapport aux milliers de kilomètres en distances nord-sud et
est-ouest. Ainsi, même si les pilotes d'avion voient réellement la courbure de la Terre, ses effets affectent
peu les distances mesurées entre les villes; notre problème peut donc être traité en 2-D!

<!-- #endregion -->



In [ ]:
n = np.arange(1, 11)
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(n, pca.singular_values_)
ax.scatter(n, pca.singular_values_, c="k", marker="*", s=60)
ax.set_xlabel("Nombres de dimensions", fontsize=22)
ax.set_ylabel("$\lambda$", fontsize=22, rotation=0)


In [ ]:
# Sélection des deux premières composantes principales

# Recalcul des composantes principales en ne gardant que les deux premières
clf = PCA(n_components=2)
Z = clf.fit_transform(X)

# À noter que le signe de chaque composante principale est arbitraire. On va changer
# celui de la première afin de générer une figure ayant la bonne
# orientation est-ouest.
Z[:, 0] = -Z[:, 0]



<!-- #region id="fc5dd8a0" -->
# <a id=affichage-des-positions-2-d-des-villes>Affichage des positions 2-D des villes</a>
<!-- #endregion -->

<!-- #region id="0072961e" -->
Dans la figure suivante, les points jaunes désignent les villes canadiennes et les bleues, les américaines. Les îles Hawaï sont en bas à gauche, celle de Porto Rico est en bas à droite. Le point jaune le plus au nord correspond à la
base militaire Alert sur l'île d'Ellesmere au Nunavut. C'est le lieu habité le plus au nord de la planète
et c'est chez nous!

> À noter que les échelles horizontales et verticales ne sont pas les mêmes afin de faciliter la comparaison avec
la figure suivante.

<!-- #endregion -->



In [ ]:
# Trouvons les indices des villes canadiennes et américaines
canada = [i for i, val in enumerate(pays) if val]
usa = [i for i, val in enumerate(pays) if not (val)]

sns.set_style("white")
fig, ax = plt.subplots(figsize=(12, 7))
ax.scatter(Z[canada, 0], Z[canada, 1], c="yellow", s=50, edgecolors="k", label="Canada")
ax.scatter(Z[usa, 0], Z[usa, 1], c="dodgerblue", s=50, edgecolors="k", label="USA")
ax.set_xlabel("$z_{1}$  (km)", fontsize=18)
ax.set_ylabel("$z_{2}$  (km)", fontsize=18)
ax.legend()
ax.set_xlim(-7000, 3500)
ax.set_ylim(-3000, 5000)



<!-- #region id="511c54e3" -->
Comparez maintenant ce résultat avec la figure suivante montrant une image satellite de la Terre prise pendant la nuit. 

> À noter que celle-ci contient des distorsions géométriques dues à la courbure de la surface terrestre.

Les distributions des villes sont assez similaires. Les îles de Porto Rico et d'Hawaï sont bien là!

<!-- #endregion -->

<!-- #region id="260fab14" -->
<p>&nbsp;</p>

<div align="center">
    <img src= "../images/satellite-earth-photo.jpeg"  width="700" />
    <div>
    <font size="0.5">Image Source: https://geology.com/articles/satellite-photo-earth-at-night.shtml/</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region id="a15edb71" -->
# <a id=y-a-t-il-dautres-applications-possibles>Y a-t-il d'autres applications possibles?</a>
<!-- #endregion -->

<!-- #region id="ed322e6f" -->
Il y en a plein! Cette méthode de génération de jeux de données est populaire en marketing.
On demande ainsi à un grand nombre de personnes d'évaluer la similarité entre un ensemble de produits.
La mesure de similarité doit être déterminée par les spécialistes du domaine.
<!-- #endregion -->

<!-- #region id="074fbb6b" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/parfume-illustration.jpeg"  width="400" />
    <div>
    <font size="0.5">Image Source: http://lamercadelplaneta1.blogspot.com/2016/04/los-niveles-de-un-producto.html/</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="f53e4c72" -->
Prenons l'exemple de parfums. On demande à des personnes de déterminer la similarité entre deux parfums
en utilisant une échelle entre 0 (totalement différents) et 5 (identiques). La distance entre les deux
parfums est donc


- $\text{distance} = 5 - \text{similarité}$,
- $\text{distance} = f(5 - \text{similarité})$,
- etc.,

où f() est une fonction de calibrage quelconque.

On fait la même chose avec tous les couples de parfums et on moyenne l'ensemble des résultats obtenus
par tous les participants afin d'obtenir de meilleures statistiques.

L'analyse de la matrice des distances, au moyen de la PCA, peut alors nous indiquer, par exemple, que les données sont
distribuées dans un espace en 4-D. Les quatre composantes de cet espace forment ce qu'on appelle des
variables latentes (c.-à-d. cachées). Certaines sont culturelles, les autres sont physiologiques (odorat).

Déterminer la nature de ces variables est intéressant en soi.
L'autre intérêt est que tous les parfums étudiés peuvent maintenant être représentés dans un espace 4-D, ce qui nous permet
de faire des regroupements de données! On peut alors identifier les différentes familles de parfums, basé sur nos réponses
culturelles/physiologiques et non sur celles attendues des spécialistes en marketing ou des spécialistes
humains (les nez en parfumerie).

De façon plus générale, ce type d'approche est utilisée en étude des marchés lorsque l'on cherche:

- Le nombre de facteurs qui affectent réellement notre jugement (les psychologues adorent).
- Les regroupements naturels des produits (les gens en marketing adorent).
<!-- #endregion -->
